In [1]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
print(tf.__version__)

2024-06-23 07:50:08.567955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 07:50:08.568049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 07:50:08.673140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0


In [3]:
pip install num2words

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.7 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [16]:
# basics
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# text_processing
from num2words import num2words
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# data preparation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# TensorFlow and Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dropout, BatchNormalization, Attention, concatenate, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2

# other
from tqdm import tqdm
import gc

# warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Load the dataset
TRAIN_DATA_PATH = '/kaggle/input/quoradata/train.csv'
df = pd.read_csv(TRAIN_DATA_PATH)


In [7]:
# Fill missing values
df['question1'] = df['question1'].fillna('')
df['question2'] = df['question2'].fillna('')

In [8]:
# Preprocessing functions
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = ' '.join([stemmer.stem(word) for word in text.split() if word not in stop_words])
    return text

# Clean text
df['question1'] = df['question1'].apply(clean_text)
df['question2'] = df['question2'].apply(clean_text)

In [10]:
# Tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(pd.concat([df['question1'], df['question2']]))

def tokenize_and_pad(text_series, max_len):
    sequences = tokenizer.texts_to_sequences(text_series)
    return pad_sequences(sequences, maxlen=max_len, padding='post')

In [11]:
max_len = 50

X_q1 = tokenize_and_pad(df['question1'], max_len)
X_q2 = tokenize_and_pad(df['question2'], max_len)

In [12]:
# GloVe embeddings
embedding_index = {}
GLOVE_PATH = '/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.200d.txt'
with open(GLOVE_PATH, encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [13]:
word_index = tokenizer.word_index
embedding_dim = 200
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [17]:
# Model definition
def create_lstm_model(embedding_matrix, max_len):
    embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                                output_dim=embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_len,
                                trainable=True)

    question1_input = Input(shape=(max_len,), name='question1_input')
    question2_input = Input(shape=(max_len,), name='question2_input')

    embedded_question1 = embedding_layer(question1_input)
    embedded_question2 = embedding_layer(question2_input)

    lstm_layer1 = Bidirectional(LSTM(32, return_sequences=True))
    lstm_layer2 = Bidirectional(LSTM(16, return_sequences=True))

    lstm_question1 = lstm_layer1(embedded_question1)
    lstm_question2 = lstm_layer1(embedded_question2)

    dropout1 = Dropout(0.2)(lstm_question1)
    dropout2 = Dropout(0.2)(lstm_question2)

    bn1 = BatchNormalization()(dropout1)
    bn2 = BatchNormalization()(dropout2)

    lstm_question1 = lstm_layer2(bn1)
    lstm_question2 = lstm_layer2(bn2)

    attention1 = Attention()([lstm_question1, lstm_question2])
    attention2 = Attention()([lstm_question2, lstm_question1])

    merged_output = concatenate([attention1, attention2])
    flatten_output = Flatten()(merged_output)
    output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01))(flatten_output)

    model = Model(inputs=[question1_input, question2_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [18]:
model = create_lstm_model(embedding_matrix, max_len)

# Split data
X = np.concatenate([X_q1, X_q2], axis=1)
y = df['is_duplicate']

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape inputs for the model
X_train_q1 = X_train[:, :max_len]
X_train_q2 = X_train[:, max_len:]
X_val_q1 = X_val[:, :max_len]
X_val_q2 = X_val[:, max_len:]


In [29]:
# Train the model
model.fit([X_train_q1, X_train_q2], y_train, epochs=30, batch_size=512, validation_data=([X_val_q1, X_val_q2], y_val))


Epoch 1/30
632/632 [==============================] - 25s 40ms/step - loss: 0.1892 - accuracy: 0.9241 - val_loss: 0.5085 - val_accuracy: 0.8028
Epoch 2/30
632/632 [==============================] - 24s 39ms/step - loss: 0.1799 - accuracy: 0.9284 - val_loss: 0.4876 - val_accuracy: 0.8211
Epoch 3/30
632/632 [==============================] - 24s 38ms/step - loss: 0.1725 - accuracy: 0.9321 - val_loss: 0.5356 - val_accuracy: 0.7963
Epoch 4/30
632/632 [==============================] - 24s 38ms/step - loss: 0.1659 - accuracy: 0.9349 - val_loss: 0.5127 - val_accuracy: 0.8100
Epoch 5/30
632/632 [==============================] - 24s 38ms/step - loss: 0.1589 - accuracy: 0.9384 - val_loss: 0.5044 - val_accuracy: 0.8310
Epoch 6/30
632/632 [==============================] - 24s 38ms/step - loss: 0.1530 - accuracy: 0.9410 - val_loss: 0.5178 - val_accuracy: 0.8294
Epoch 7/30
632/632 [==============================] - 24s 38ms/step - loss: 0.1481 - accuracy: 0.9430 - val_loss: 0.5496 - val_accuracy:

In [30]:
# Predict on validation set
y_pred_val = (model.predict([X_val_q1, X_val_q2]) > 0.5).astype("int32")

2527/2527 [==============================] - 19s 8ms/step


In [31]:
# Print performance metrics
accuracy = accuracy_score(y_val, y_pred_val)
report = classification_report(y_val, y_pred_val)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.824828712063123
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.82      0.85     50803
           1       0.73      0.83      0.78     30055

    accuracy                           0.82     80858
   macro avg       0.81      0.83      0.82     80858
weighted avg       0.83      0.82      0.83     80858

